## Standardized test cases

For benchmarking, we need a standardized set of case studies, ranging form simple to very large. This helps us to test and track the performance of improvements of bw_timex with the same set of test cases.


### 0) set up variables & functions

In [1]:
import numpy as np
import bw2data as bd

from bw_temporalis import TemporalDistribution
from datetime import datetime

from cProfile import Profile
from pstats import SortKey, Stats

In [2]:
assert 'timex' in bd.projects
if 'timex' not in bd.projects:
    import bw2io as bi
    bi.backup.restore_project_directory(fp= 'your/path/to/file/backupfile.tar.gz',
                                    overwrite_existing=True)

In [3]:

bd.databases

Databases dictionary with 1 object(s):
	biosphere3

In [4]:
bd.projects.set_current("timex")
to_delete = []
for db in bd.databases: #deleting all test_systems
    if "test" in db:
        to_delete.append(db)
print(to_delete)

for db in to_delete:
    del bd.databases[db] 

print(bd.databases)

['test_small']
Databases dictionary with 7 object(s):
	biosphere
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff
	ei310_IMAGE_SSP2_RCP19_2020_electricity
	ei310_IMAGE_SSP2_RCP19_2030_electricity
	ei310_IMAGE_SSP2_RCP19_2040_electricity
	foreground


In [5]:
base_background_db = bd.Database("ei310_IMAGE_SSP2_RCP19_2020_electricity")

In [6]:
# list_random_acts = []
# for i in range(0, 100):
#     act = base_background_db.random()
#     list_random_acts.append((act["code"]))

In [7]:
#created by Amelie from cell above
fixed_random_acts = ['413d39ccf7df41e0ab3ba3235f2a18b3', '61509bd071a4fda35bdff55d9099bd10', 'fe3220b739ec4e8fb31c349cbc3e7592', '366a6d4bc20340d887492ca37ef05e44', 'd381c55db382e2a32a61b5052ac9e0bc', 'a72e756d6bf75cd2261ff8a85f5e8d7d', '368183f3747ff59590ae360f26c41938', 'e0337f899fc9a5a335d0d34059d073a7', '859bdf006eb63da45068d07d628fe2d0', '6c46344dba484b688b81724b94f892fb', '718efefd501788a8e7d34091c142f5bf', 'c5ce453090e24917b2d20a2b5401ceea', '6b458afa4e04a34ee1eab72e2187db57', '7c02589d38a8ebe7730b84c69cced834', 'cfb6a5fd92605adcdf8ac016117b1bf2', '726a73f728c2f40a1c38a7a8818fd561', '5043bfcd8aa2f8f9ece4993e399ab2e5', '710ebcc0ef46858b11fa6a8d9855659e', '12abcffb21749f413998f92dcd9e3df6', '5c9c7c2df92a4113913d422f2c391ae7', '7be9d1fe9ad573a4a65d64d0f2918a3d', '75575808b29649587a25c44a5fa1fb44', 'de8f562bc09d360c74a364698991bf2d', '5df53982de25cec816af740f9771fbe8', 'ec6af463696c6dcfd36aeecfea3db8f1', '2b6e9470628e82e012e10cf2914f2fce', 'gaas_4', '022c223c1a6f4f609ee1b3b5488b1727', '6ddd150394f3485c27947e82e5d67b2b', '35e4bb257b5da0271d02c2cc51560dcd', '82b28d684680d09d636bae5b936c9b3e', '82edb367427f6bd1c2c66a85295b42bb', '35ebbd3df417434eab26b06154f86ede', 'ac4aed5238212a782199463386c1a7da', '3684b170cdad51130ac697e254ed832c', '5d9a11ca930c3246279972f27ef21470', '07d9c106a8585cde1e07f222ee20de55', 'e65d3ba6312a5895324697668e16e0c3', '695fe6ceb13b3fd67be725eb9dc37794', '1e805a4fff6e93965cc5a86445045953', '7da536778d811cb6fbb6da7da56d9552', '54b2952ac69454c9eb7c2e5610508ad7', 'e28f3fcd5f31c4b88c29f0c2582278f4', '5218269bfd071b1a25aa04479068a1c1', '5579a190a05086c1c71841194e0ad8fe', 'cc16405a6a2d431f86ce0133b1e7e7a0', 'a87bbdc22b3032ba3592d20a489d322c', 'fcfdb0039efc325f23c7b796f8e99bd7', '8499425615f15e222d3f351a80541237', 'adabde2043e70876877d022ba602b3ae', 'e2fad698f73847f6f485d61ed4ab5ccc', 'c845df8016ef1945e725052cdb8e5628', 'c9d8ee97d295790ff15482a88088ef09', '4cb938b8a49683e82d621c4708810245', 'b39303d7f4595ca157599c928bf637d4', 'd29ea81dd6694927b10fb63f43405319', '33cdd5ae4a864de8a5d8e96944a3c0d9', '3e4636c941fead54e9098f002b916d20', 'deeb9755c7fb9e34d9527f92281a4036', 'dbc7a6cdeb62474fafe298887d3ab762', 'edf07b8e432b303dd815685ee43bcabb', 'a023cb6dfa8a43b5adb64bd6fe817948', '4d4b4baf68e84acf5ab531b00ce11b97', '9d50b22e553048fdb03bd51860827471', '200b9ec67588ca3b72b846f9e2f2ac6c', '03db8f023142a84ee971ca3b8f496056', '9dfcba1a3531f34c8b5f78238e2abad2', '241b6545f79c4a27a4b249091885e748', '82abfe77a5acdd9bebf4a4ccabccd7c3', '52c6ddfa5a472219c7bfccefc6e0abda', 'cf282afd70d1f2e0827296e8d8f7793d', 'c0325f3b5f3e1351527a96505d5f8627', 'fb7c379849825156878685aee1c397e6', 'a75b64db7dbd0bd03a29ce7312fc73e4', 'ee6c982ed16f0408090bc21ff57dfcb1', '0dcc429b44d867ed8f97e825849635e5', 'bd43a26a3e741cf79d5c8302de1b6183', 'e78e7465d2883bf9761060f02a57d347', '39663eb7360e4bff044fe364101c77d7', 'cd8731d74dd630036ccbb9ec8093a765', 'a8f4e24a673115218914d22e1d5f56e9', 'dc75be81bfeadad7108da983e0892517', '4019ccc70cb4aef39c6ac9754a982bcf', 'b835dee82350b353d2fee88daa07cb12', '6ddb258254a59b65cdffdc4d3d9a8173', '42472d9cb7a843cd4da22fec38844569', 'd17ed61046a64649863656b7d6803c3f', '9c584ad05905021f8b1d9014ffa47faa', 'ae4c7c7f021e4601a4f9b4e6d7749d5d', 'be47eee7e8baad5ec086ce3faa750057', '2a54b0efec6081a21f28e5d872f39718', '2fb1edc5fae96916c608bd7c8c2a22c2', 'ed9de9cc8bbebcc7cb2d18ee76292c55', '8f65daaa9301ffabbe53800495390ad4', '49efabed4e7747916d6e42b5e27651bb', '99b425f7e13fdf832ac1c128aff0642f', 'b6a457b4716a824577e9eb9ff2daf28a', '15516091e07004de2f275891dc980671', '9c4433c53c6413d47737f6f2011c9859', 'a94b3446d67eaaed30eef42513790567']

In [8]:
for code in fixed_random_acts[:4]: #check the nodes
    node = base_background_db.get(code = code)
    print(node)

'light duty truck, battery electric, 7.5t gross weight, long haul' (unit, RER, None)
'market for oriented strand board' (cubic meter, RER, None)
'Medium duty truck, diesel, 26t gross weight, EURO-IV, long haul' (unit, RER, None)
'market group for electricity, low voltage, 20-year period' (kilowatt hour, INDIA, None)


In [9]:
def create_standardized_test_system(db_name, number_tiers, number_processes_per_tier, td_timesteps, td_dtype, fixed_random_acts, number_background_exchanges_per_process, loop = False):
    """
    Create a foreground database with a standardized structure for testing purposes. It contains number_tiers supply chain levels, with each containing 
    number_processes_per_tier processes. Each process consumes from all processes in the tier below. At the last tier, each process consumes a certain numebr of background process
    Each process has a temporal distribution with td_timesteps steps, e.g. 3, in td_dtype resolution, e.g. years, starting at a unit of time later than the consuming process
    The last tier consumes a process from the background system in 2020, selecting a process from a predefined list of random processes, with the same temporal distribution as above.
    If loop = True, a loop is included: the last process in last tier consumes from the last process in first tier, no TDs at this node.
    No additional TDs at biosphere flows.

    Parameters:
    db_name: str, name of the database
    number_tiers: int, number of tiers in the supply chain
    number_processes_per_tier: int, number of processes per tier
    td_timesteps: int, number of timesteps in the temporal distribution
    td_dtype: str, dtype of the temporal distribution, e.g. "timedelta64[Y]" or "timedelta64[M]"
    fixed_random_acts: a list of 100 background activities codes that has been pre-generated
    number_background_exchanges_per_process: int, number of background exchanges per process in the last tier
    loop: bool, whether to include a loop in the foreground system

    returns: None

    
    """
    
    # create a new database
    if db_name in bd.databases:
        del bd.databases[db_name]
        print(f"Database {db_name} already exists, overwriting.")
    foreground = bd.Database(db_name)
    foreground.register()

    background_db = bd.Database("ei310_IMAGE_SSP2_RCP19_2020_electricity") # background database for 2020 in timex project

    # save FU node
    FU_node = foreground.new_node(
        code="FU",
        name="functional unit",
        unit = "unit"
    )
    FU_node.save()
    FU_node.new_edge(input=FU_node , amount=1, unit = "unit", type="production").save()

    #create nodes
    for tier in range(1, number_tiers+1):
        for process in range(1, number_processes_per_tier+1):
            process_name = f'tier{tier}_act{process}'
            process_code = f't{tier}_a{process}'
            process_node = foreground.new_node(
                code=process_code,
                name=process_name,
                unit = "unit"
            )
            process_node.save()
            process_node.new_edge(input=process_node , amount=1, unit = "unit", type="production").save()

    #relink nodes
    for tier in range(1, number_tiers+1):
        for process in range(1, number_processes_per_tier+1):
            process_name = f'tier{tier}_act{process}'
            process_code = f't{tier}_a{process}'
            process_node = foreground.get(process_code)
            if tier == 1: #tier 1 = FU
                consuming_node = FU_node
                new_edge = consuming_node.new_edge(input=process_node , amount=1, unit = "unit", type="technosphere")
                new_edge.save()
            else: # lower tiers: all nodes at the tier above consume from all nodes at the tier below, with a temporal distribution specified by the inputs
                for process in range(1, number_processes_per_tier+1):
                    consuming_node = foreground.get(f't{tier-1}_a{process}')
                    new_edge = consuming_node.new_edge(input=process_node , amount=1, unit = "unit", type="technosphere")
                    temporal_distribution = TemporalDistribution(date = np.array(np.arange(1, td_timesteps+1), dtype = td_dtype), 
                                                                amount = np.full(td_timesteps, 1 / td_timesteps)) #linear distribution between 1 and td_timesteps in years
                    new_edge["temporal_distribution"] = temporal_distribution
                    new_edge.save()

    # add background process to nodes on last tier
    count_background_act = 0 #counting upward in list of random acts
    total_background_count = 0
    for process in range(1, number_processes_per_tier+1):
        last_foreground_node = foreground.get(f't{number_tiers}_a{process}')

        for _ in range (1, number_background_exchanges_per_process+1):
            background_process = background_db.get(code = fixed_random_acts[count_background_act])
            count_background_act += 1
            
            new_edge = last_foreground_node.new_edge(input=background_process, amount=4, type="technosphere")
            temporal_distribution = TemporalDistribution(date = np.array(np.arange(1, td_timesteps+1), dtype = td_dtype),
                                                            amount = np.full(td_timesteps, 1 / td_timesteps)) #linear distribution between 1 and td_timesteps in years
            new_edge["temporal_distribution"] = temporal_distribution
            new_edge.save()

            if count_background_act == 99: # if all 100 background processes have been used, start again
                total_background_count +=count_background_act 
                count_background_act = 0

    if loop:
        # add loop to foreground: last process in last tier consumes from last process in first tier, no TDs
        loop_consumer = foreground.get(f't{number_tiers}_a{number_processes_per_tier}')
        loop_producer = foreground.get(f't{1}_a{number_processes_per_tier}')
        new_edge = loop_consumer.new_edge(input=loop_producer, amount=0.3, type="technosphere")
        new_edge.save()

    count_exc = 0
    for act in foreground:
        for exc in act.technosphere():
            count_exc += 1

    total_background_count +=count_background_act #number of background processes used in total
            
    print(f"Database {db_name} created with {len(foreground)} processes and {count_exc} exchanges. There are {total_background_count} different background processes used.")

### 1) small system

In [10]:
db_name = "test_small"
create_standardized_test_system(db_name = db_name, number_tiers = 2 , number_processes_per_tier=2,
                                td_timesteps= 2, td_dtype = "timedelta64[Y]", fixed_random_acts= fixed_random_acts, number_background_exchanges_per_process = 2, loop = False)
                               

Database test_small created with 5 processes and 10 exchanges. There are 4 different background processes used.


In [11]:
# check db entries:
foreground = bd.Database(db_name)
for act in foreground:
    print(act)
    for exc in act.exchanges():
        td = exc.get("temporal_distribution", None)
        print("--", exc["type"],"|", exc,"|", td)  
    print ("\n")

'tier2_act2' (unit, GLO, None)
-- production | Exchange: 1 unit 'tier2_act2' (unit, GLO, None) to 'tier2_act2' (unit, GLO, None)> | None
-- technosphere | Exchange: 4 unit 'Medium duty truck, diesel, 26t gross weight, EURO-IV, long haul' (unit, RER, None) to 'tier2_act2' (unit, GLO, None)> | TemporalDistribution instance with 2 values and total: 1
-- technosphere | Exchange: 4 kilowatt hour 'market group for electricity, low voltage, 20-year period' (kilowatt hour, INDIA, None) to 'tier2_act2' (unit, GLO, None)> | TemporalDistribution instance with 2 values and total: 1


'tier2_act1' (unit, GLO, None)
-- production | Exchange: 1 unit 'tier2_act1' (unit, GLO, None) to 'tier2_act1' (unit, GLO, None)> | None
-- technosphere | Exchange: 4 unit 'light duty truck, battery electric, 7.5t gross weight, long haul' (unit, RER, None) to 'tier2_act1' (unit, GLO, None)> | TemporalDistribution instance with 2 values and total: 1
-- technosphere | Exchange: 4 cubic meter 'market for oriented strand 

In [12]:
database_dates = {
    "ei310_IMAGE_SSP2_RCP19_2020_electricity": datetime.strptime("2020", "%Y"),
    "ei310_IMAGE_SSP2_RCP19_2030_electricity": datetime.strptime("2030", "%Y"),
    "ei310_IMAGE_SSP2_RCP19_2040_electricity": datetime.strptime("2040", "%Y"),
    db_name: "dynamic",
}

In [13]:
from bw_timex import TimexLCA
FU_node = bd.Database(db_name).get("FU")

with Profile() as profile:
    tlca = TimexLCA(
        demand={FU_node: 1},
        method=("EF v3.1", "climate change", "global warming potential (GWP100)"),
        database_dates=database_dates,
    )
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         3066698 function calls (3062532 primitive calls) in 6.984 seconds

   Ordered by: cumulative time
   List reduced from 1069 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    6.984    6.984 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:91(__init__)
        1    0.000    0.000    3.457    3.457 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:384(lci)
        1    0.000    0.000    3.007    3.007 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:411(lci_calculation)
        1    0.000    0.000    2.945    2.945 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:366(solve_linear_system)
        1    0.000    0.000    2.945    2.945 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\s

In [14]:
with Profile() as profile:
    tlca.build_timeline()
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:208: UserWarning: No edge filter function provided. Skipping all edges in background databases.
  warnings.warn(


Starting graph traversal
Calculation count: 14
         7027143 function calls (6772851 primitive calls) in 11.147 seconds

   Ordered by: cumulative time
   List reduced from 1534 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    81089    0.097    0.000    6.676    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:4664(next)
    81061    0.133    0.000    6.578    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:4578(iterate)
    80869    0.315    0.000    4.413    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:7894(process_row)
    80869    0.360    0.000    2.690    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:7852(process_row)
  194/192    0.000    0.000    2.390    0.012 c:\Users\MULLERA\AppData\Local\anacon

In [15]:
with Profile() as profile:
    tlca.lci()
    
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         6964408 function calls (6869027 primitive calls) in 18.994 seconds

   Ordered by: cumulative time
   List reduced from 1404 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.000    0.000   18.995   18.995 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:268(lci)
      107    0.004    0.000    9.804    0.092 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2data\utils.py:362(get_node)
        1    0.014    0.014    9.493    9.493 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:567(calculate_dynamic_inventory)
        1    0.387    0.387    9.247    9.247 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\dynamic_biosphere_builder.py:102(build_dynamic_biosphere_matrix)
  199/154    0.000    0.000    7.517    0.049 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_

In [16]:
with Profile() as profile:
    tlca.static_lcia()

Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         2393 function calls (2392 primitive calls) in 0.005 seconds

   Ordered by: cumulative time
   List reduced from 187 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.000    0.000    0.005    0.005 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:360(static_lcia)
        1    0.000    0.000    0.004    0.004 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:283(load_lcia_data)
        1    0.000    0.000    0.004    0.004 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\matrix_utils\mapped_matrix.py:15(__init__)
        2    0.000    0.000    0.001    0.001 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\matrix_utils\aggregation.py:5(aggregate_with_sparse)
        1    0.000    0.000    0.001    0.001 c:\Users\MULLERA\AppData\Local\anaconda3\envs\

In [17]:
tlca.static_score

731929.2292028617

### 2) medium system

In [18]:
# needed to avoid multiple results error https://github.com/brightway-lca/bw_timex/issues/101
to_delete = []
for db in bd.databases: #deleting all test_systems
    if "test" in db:
        to_delete.append(db)
print(to_delete)

for db in to_delete:
    del bd.databases[db] 

print(bd.databases)

['test_small']
Databases dictionary with 7 object(s):
	biosphere
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff
	ei310_IMAGE_SSP2_RCP19_2020_electricity
	ei310_IMAGE_SSP2_RCP19_2030_electricity
	ei310_IMAGE_SSP2_RCP19_2040_electricity
	foreground


In [19]:
db_name = "test_medium"
create_standardized_test_system(db_name = db_name, number_tiers = 4, number_processes_per_tier=4,
                                td_timesteps= 2, td_dtype = "timedelta64[Y]", fixed_random_acts= fixed_random_acts, number_background_exchanges_per_process = 4,
                                loop = False)

Database test_medium created with 17 processes and 68 exchanges. There are 16 different background processes used.


In [ ]:
# check db entries:
# foreground = bd.Database(db_name)
# for act in foreground:
#     print(act)
#     for exc in act.exchanges():
#         td = exc.get("temporal_distribution", None)
#         print("--", exc["type"],"|", exc,"|", td)  
#     print ("\n")

'tier2_act4' (unit, GLO, None)
-- production | Exchange: 1 unit 'tier2_act4' (unit, GLO, None) to 'tier2_act4' (unit, GLO, None)> | None
-- technosphere | Exchange: 1 unit 'tier3_act1' (unit, GLO, None) to 'tier2_act4' (unit, GLO, None)> | TemporalDistribution instance with 2 values and total: 1
-- technosphere | Exchange: 1 unit 'tier3_act2' (unit, GLO, None) to 'tier2_act4' (unit, GLO, None)> | TemporalDistribution instance with 2 values and total: 1
-- technosphere | Exchange: 1 unit 'tier3_act3' (unit, GLO, None) to 'tier2_act4' (unit, GLO, None)> | TemporalDistribution instance with 2 values and total: 1
-- technosphere | Exchange: 1 unit 'tier3_act4' (unit, GLO, None) to 'tier2_act4' (unit, GLO, None)> | TemporalDistribution instance with 2 values and total: 1


'tier1_act4' (unit, GLO, None)
-- production | Exchange: 1 unit 'tier1_act4' (unit, GLO, None) to 'tier1_act4' (unit, GLO, None)> | None
-- technosphere | Exchange: 1 unit 'tier2_act1' (unit, GLO, None) to 'tier1_act4' (u

In [21]:
database_dates = {
    "ei310_IMAGE_SSP2_RCP19_2020_electricity": datetime.strptime("2020", "%Y"),
    "ei310_IMAGE_SSP2_RCP19_2030_electricity": datetime.strptime("2030", "%Y"),
    "ei310_IMAGE_SSP2_RCP19_2040_electricity": datetime.strptime("2040", "%Y"),
    db_name: "dynamic",
}

In [22]:
from bw_timex import TimexLCA
FU_node = bd.Database(db_name).get("FU")

with Profile() as profile:
    tlca = TimexLCA(
        demand={FU_node: 1},
        method=("EF v3.1", "climate change", "global warming potential (GWP100)"),
        database_dates=database_dates,
    )
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         3200943 function calls (3190201 primitive calls) in 8.003 seconds

   Ordered by: cumulative time
   List reduced from 969 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    7.999    7.999 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:91(__init__)
        1    0.000    0.000    3.731    3.731 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:384(lci)
        1    0.033    0.033    3.280    3.280 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:986(create_node_collections)
        1    0.000    0.000    3.196    3.196 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:411(lci_calculation)
        1    0.000    0.000    3.140    3.140 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\

In [23]:
with Profile() as profile:
    tlca.build_timeline()
    
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10) #1 min

C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:208: UserWarning: No edge filter function provided. Skipping all edges in background databases.
  warnings.warn(


Starting graph traversal
Calculation count: 1300
         16013541 function calls (15410512 primitive calls) in 29.258 seconds

   Ordered by: cumulative time
   List reduced from 1505 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.008    0.008   13.279   13.279 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timeline_builder.py:29(__init__)
    94373    0.113    0.000    9.522    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:4664(next)
    91773    0.161    0.000    9.409    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:4578(iterate)
     5548    0.028    0.000    8.290    0.001 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:2260(_execute)
     5548    0.020    0.000    8.197    0.001 c:\Users\MULLERA\AppData\Local\anaconda3\envs

In [24]:
with Profile() as profile:
    tlca.lci()
    
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10) #37 secs

         44817463 function calls (44615308 primitive calls) in 173.791 seconds

   Ordered by: cumulative time
   List reduced from 1427 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1078    0.045    0.000  131.225    0.122 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2data\utils.py:362(get_node)
2320/1717    0.005    0.000  119.338    0.070 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:2012(inner)
     1717    0.002    0.000  119.312    0.069 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:2087(execute)
     1514    0.003    0.000  119.116    0.079 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:7254(__iter__)
     1717    0.016    0.000  116.531    0.068 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-pa

In [25]:
with Profile() as profile:
    tlca.static_lcia()

Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         5725 function calls (5724 primitive calls) in 0.008 seconds

   Ordered by: cumulative time
   List reduced from 187 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.000    0.000    0.008    0.008 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:360(static_lcia)
        1    0.000    0.000    0.008    0.008 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:424(lcia)
        1    0.000    0.000    0.004    0.004 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:283(load_lcia_data)
       25    0.003    0.000    0.004    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw_processing\datapackage.py:180(filter_by_attribute)
        1    0.000    0.000    0.003    0.003 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_

In [26]:
tlca.static_score

140053819.69744948

### 3) large system


In [27]:
# needed to avoid multiple results error https://github.com/brightway-lca/bw_timex/issues/101
to_delete = []
for db in bd.databases: #deleting all test_systems
    if "test" in db:
        to_delete.append(db)
print(to_delete)

for db in to_delete:
    del bd.databases[db] 

print(bd.databases)

['test_medium']
Databases dictionary with 7 object(s):
	biosphere
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff
	ei310_IMAGE_SSP2_RCP19_2020_electricity
	ei310_IMAGE_SSP2_RCP19_2030_electricity
	ei310_IMAGE_SSP2_RCP19_2040_electricity
	foreground


In [36]:
db_name = "test_large"
create_standardized_test_system(db_name = db_name, number_tiers = 5, number_processes_per_tier=4,
                                td_timesteps= 2, td_dtype = "timedelta64[Y]", fixed_random_acts= fixed_random_acts, number_background_exchanges_per_process = 10,
                                loop = False)

Database test_large already exists, overwriting.
Database test_large created with 21 processes and 108 exchanges. There are 40 different background processes used.


In [29]:
# # check db entries:
# foreground = bd.Database(db_name)
# for act in foreground:
#     print(act)
#     for exc in act.exchanges():
#         td = exc.get("temporal_distribution", None)
#         print("--", exc["type"],"|", exc,"|", td)  
#     print ("\n")

In [37]:
database_dates = {
    "ei310_IMAGE_SSP2_RCP19_2020_electricity": datetime.strptime("2020", "%Y"),
    "ei310_IMAGE_SSP2_RCP19_2030_electricity": datetime.strptime("2030", "%Y"),
    "ei310_IMAGE_SSP2_RCP19_2040_electricity": datetime.strptime("2040", "%Y"),
    db_name: "dynamic",
}

In [38]:
from bw_timex import TimexLCA
FU_node = bd.Database(db_name).get("FU")

with Profile() as profile:
    tlca = TimexLCA(
        demand={FU_node: 1},
        method=("EF v3.1", "climate change", "global warming potential (GWP100)"),
        database_dates=database_dates,
    )
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         3276515 function calls (3261511 primitive calls) in 10.271 seconds

   Ordered by: cumulative time
   List reduced from 1018 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   151/87    0.007    0.000   18.335    0.211 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\queue.py:154(get)
      167    0.101    0.001   10.982    0.066 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\threading.py:323(wait)
   112569    0.278    0.000    4.750    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:4664(next)
   112569    0.367    0.000    4.272    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:4578(iterate)
26918/26917    0.271    0.000    4.224    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2data\backends\base.py:316(__iter__)

In [39]:
with Profile() as profile:
    tlca.build_timeline()
    
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10) #43 sec

C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:208: UserWarning: No edge filter function provided. Skipping all edges in background databases.
  warnings.warn(


Starting graph traversal


c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw_graph_tools\graph_traversal.py:473: UserWarning: Stopping traversal due to calculation count.
  warnings.warn("Stopping traversal due to calculation count.")


Calculation count: 2003
         20016303 function calls (19340280 primitive calls) in 53.484 seconds

   Ordered by: cumulative time
   List reduced from 1501 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  784/444    0.090    0.000   72.284    0.163 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\queue.py:154(get)
2004/2003    0.021    0.000   38.254    0.019 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\bw_temporalis\bw_temporalis\lca.py:314(get_technosphere_exchange)
     2003    0.143    0.000   30.114    0.015 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\bw_temporalis\bw_temporalis\lca.py:290(_exchange_iterator)
8125/8123    0.028    0.000   19.389    0.002 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:2012(inner)
     8123    0.021    0.000   19.361    0.002 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\sit

In [40]:
with Profile() as profile:
    tlca.lci()
    
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         36717253 function calls (36448066 primitive calls) in 254.859 seconds

   Ordered by: cumulative time
   List reduced from 1404 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
3782/2156    4.570    0.001  465.395    0.216 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\queue.py:154(get)
     1627    2.331    0.001  351.015    0.216 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\threading.py:637(wait)
     4017    7.364    0.002  219.295    0.055 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\threading.py:323(wait)
     1412    0.110    0.000  171.965    0.122 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2data\utils.py:362(get_node)
3442/2455    0.015    0.000  164.844    0.067 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:2012(inner)
     2455    0.007  

In [41]:
with Profile() as profile:
    tlca.static_lcia()

Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         6226 function calls in 0.016 seconds

   Ordered by: cumulative time
   List reduced from 213 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.016    0.016 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:424(lcia)
        1    0.000    0.000    0.016    0.016 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:446(lcia_calculation)
        1    0.000    0.000    0.016    0.016 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\scipy\sparse\_matrix.py:43(__mul__)
       25    0.008    0.000    0.009    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw_processing\datapackage.py:180(filter_by_attribute)
        1    0.000    0.000    0.009    0.009 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_l

In [42]:
tlca.static_score

538237164.3091656

### 4) small system with loop

In [43]:
# needed to avoid multiple results error https://github.com/brightway-lca/bw_timex/issues/101
to_delete = []
for db in bd.databases: #deleting all test_systems
    if "test" in db:
        to_delete.append(db)
print(to_delete)

for db in to_delete:
    del bd.databases[db] 

print(bd.databases)

['test_large']
Databases dictionary with 7 object(s):
	biosphere
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff
	ei310_IMAGE_SSP2_RCP19_2020_electricity
	ei310_IMAGE_SSP2_RCP19_2030_electricity
	ei310_IMAGE_SSP2_RCP19_2040_electricity
	foreground


In [44]:

db_name = "test_loop"
create_standardized_test_system(db_name = db_name, number_tiers = 2, number_processes_per_tier=3,
                                td_timesteps= 2, td_dtype = "timedelta64[Y]",  fixed_random_acts= fixed_random_acts, number_background_exchanges_per_process = 2, 
                                loop = True)

Database test_loop created with 7 processes and 19 exchanges. There are 6 different background processes used.


In [45]:
# # check db entries:
# foreground = bd.Database(db_name)
# for act in foreground:
#     print(act)
#     for exc in act.exchanges():
#         td = exc.get("temporal_distribution", None)
#         print("--", exc["type"],"|", exc,"|", td)  
#     print ("\n")

In [46]:
database_dates = {
    "ei310_IMAGE_SSP2_RCP19_2020_electricity": datetime.strptime("2020", "%Y"),
    "ei310_IMAGE_SSP2_RCP19_2030_electricity": datetime.strptime("2030", "%Y"),
    "ei310_IMAGE_SSP2_RCP19_2040_electricity": datetime.strptime("2040", "%Y"),
    db_name: "dynamic",
}

In [47]:
from bw_timex import TimexLCA
FU_node = bd.Database(db_name).get("FU")

with Profile() as profile:
    tlca = TimexLCA(
        demand={FU_node: 1},
        method=("EF v3.1", "climate change", "global warming potential (GWP100)"),
        database_dates=database_dates,
    )
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         3085519 function calls (3079666 primitive calls) in 9.314 seconds

   Ordered by: cumulative time
   List reduced from 1023 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      153    0.088    0.001   16.859    0.110 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\threading.py:323(wait)
   134/76    0.006    0.000   16.033    0.211 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\queue.py:154(get)
        1    0.000    0.000    4.934    4.934 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:91(__init__)
   112087    0.253    0.000    4.470    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:4664(next)
   112087    0.334    0.000    4.215    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:4578(iterate)
    26903    0.224    0.000  

In [48]:
with Profile() as profile:
    tlca.build_timeline()
    
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:208: UserWarning: No edge filter function provided. Skipping all edges in background databases.
  warnings.warn(


Starting graph traversal
Calculation count: 378


C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timeline_builder.py:537: Warning: Reference date 2041-01-01 00:00:00 is higher than all provided dates. Data will be taken from the closest lower year.
  warnings.warn(
C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timeline_builder.py:537: Warning: Reference date 2042-01-01 00:00:00 is higher than all provided dates. Data will be taken from the closest lower year.
  warnings.warn(
C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timeline_builder.py:537: Warning: Reference date 2043-01-01 00:00:00 is higher than all provided dates. Data will be taken from the closest lower year.
  warnings.warn(
C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timeline_builder.py:537: Warning: Reference date 2044-01-01 00:00:00 is higher than all provided dates. Data will be taken from the closest lower year.
  warnings.warn(
C:\Users\MULLERA\OneDrive - VITO

         11872979 function calls (11417391 primitive calls) in 33.512 seconds

   Ordered by: cumulative time
   List reduced from 1520 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  464/101    0.084    0.000   23.012    0.228 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\queue.py:154(get)
      202    0.069    0.000   14.910    0.074 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\threading.py:637(wait)
    88527    0.264    0.000    9.809    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:4664(next)
    87771    0.340    0.000    9.278    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:4578(iterate)
3549/3547    0.011    0.000    8.268    0.002 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:2012(inner)
     3547    0.0

In [49]:
with Profile() as profile:
    tlca.lci()
    
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         90711622 function calls (90433195 primitive calls) in 333.622 seconds

   Ordered by: cumulative time
   List reduced from 1421 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
4847/2745    9.899    0.002  460.554    0.168 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\queue.py:154(get)
     2105    5.970    0.003  418.366    0.199 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\threading.py:637(wait)
       68    0.057    0.001  352.680    5.186 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\asyncio\base_events.py:1910(_run_once)
     5107   19.687    0.004  266.560    0.052 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\threading.py:323(wait)
     1689    0.132    0.000  140.724    0.083 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2data\utils.py:362(get_node)
3218/2516    0.01

In [50]:
with Profile() as profile:
    tlca.static_lcia()

Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         8805 function calls (8801 primitive calls) in 0.017 seconds

   Ordered by: cumulative time
   List reduced from 257 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       25    0.007    0.000    0.009    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw_processing\datapackage.py:180(filter_by_attribute)
        1    0.000    0.000    0.006    0.006 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\utils.py:19(consistent_global_index)
        1    0.000    0.000    0.005    0.005 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:283(load_lcia_data)
        1    0.000    0.000    0.004    0.004 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\matrix_utils\mapped_matrix.py:15(__init__)
        1    0.000    0.000    0.003    0.003 c:\Users\MULLERA

In [51]:
tlca.static_score

1653790.5231715888